In [1]:
# Nhập các thư viện cần thiết trong quá trình thực hiện
import vnstock as vnst
import pandas as pd
import numpy as np
import cvxpy as cp

ModuleNotFoundError: No module named 'cvxpy'

In [2]:
def get_combined_df(list_stock):    
    symbols = list_stock
    start_date = "2019-04-25"
    end_date = "2020-01-01"

    historical_data = {}

    for symbol in symbols:
        historical_data[symbol] = vnst.stock_historical_data(
            symbol=symbol,
            start_date=start_date,
            end_date=end_date,
            # type=type
        )

    # Gộp các DataFrame trong danh sách theo cột 'close'
    df_combined = pd.concat([historical_data[symbol]['close'] for symbol in symbols], axis=1)

    # Đặt tên cột mới
    df_combined.columns = symbols
    return df_combined

In [3]:
def covariance_matrix(corr_matrix, volatility):
    n = len(volatility)
    cov_matrix = np.zeros((n, n))
    for i in range(n):
        for j in range(n):
            cov_matrix[i, j] = corr_matrix.iloc[i, j] * volatility[i] * volatility[j]  # Changed corr_matrix[i, j] to corr_matrix.iloc[i, j]
    return cov_matrix

def objective(x, cov_matrix):
    return cp.quad_form(x, cov_matrix)

def cal_weight(df_combined, list_stock):
    days_return_3days = df_combined / df_combined.shift(3) - 1
    days_return_3days.dropna(inplace=True)
    # returns_20 = days_return_3days.head(20).mean()
    # returns_60 = days_return_3days.tail(60).mean()
    volatility = np.std(days_return_3days.tail(20))
    corr_matrix = df_combined.corr()

    cov_matrix = covariance_matrix(corr_matrix, volatility)
    num_assets = cov_matrix.shape[0]
    x = cp.Variable(num_assets)
    objective_func = cp.Minimize(cp.quad_form(x, cov_matrix))
    constraints = [
        cp.sum(x) == 1,  # a + b + c + d + e = 1
        x >= 0.05,  # a, b, c, d, e >= 0.05
        cp.matmul(returns_20.values, x) >= 0,  # Lớn hơn hoặc bằng 0
        cp.matmul(returns_60.values, x) >= 0  # Lớn hơn hoặc bằng 0
    ]
    problem = cp.Problem(objective_func, constraints)
    weight_df = problem.solve()
    weight_df = pd.DataFrame({'ticker': list_stock,'percentage': x.value}).round(4)
    return weight_df

In [4]:
list_stock = ['SSI','HPG','VND','GAS','FPT']
df_combined =  get_combined_df(list_stock)
weight_df = cal_weight(df_combined, list_stock)
weight_df.head()

d:\ATTACKER\Attacker-2024\env\Lib\site-packages\numpy\core\fromnumeric.py:3643: FutureWarning: The behavior of DataFrame.std with axis=None is deprecated, in a future version this will reduce over both axes and return a scalar. To retain the old behavior, pass axis=0 (or do not pass axis)
  return std(axis=axis, dtype=dtype, out=out, ddof=ddof, **kwargs)
C:\Users\FLOWER\AppData\Local\Temp\ipykernel_79456\251464999.py:6: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  cov_matrix[i, j] = corr_matrix.iloc[i, j] * volatility[i] * volatility[j]  # Changed corr_matrix[i, j] to corr_matrix.iloc[i, j]


,ticker,percentage
0,SSI,0.1230
1,HPG,0.0500
2,VND,0.3462
3,GAS,0.0500
4,FPT,0.4308


In [7]:
ad = vnst.stock_historical_data('BMP','2019-04-25','2020-01-01')
ad

,time,open,high,low,close,volume,ticker
0,2019-04-25,28710,28710,28460,28460,70400,BMP
1,2019-04-26,28430,28950,28370,28950,68280,BMP
2,2019-05-02,29010,29130,28530,28530,83180,BMP
3,2019-05-03,28530,28710,28460,28620,30040,BMP
4,2019-05-06,28580,28580,27790,27790,74640,BMP
...,...,...,...,...,...,...,...
170,2019-12-25,31440,31440,31240,31330,12440,BMP
171,2019-12-26,31310,31760,31240,31540,15880,BMP
172,2019-12-27,31500,31500,30910,31000,32460,BMP
173,2019-12-30,31000,32210,30570,30610,68450,BMP
